<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="300" alt="cognitiveclass.ai logo"  />
</center>


# Financial Services: Lab 3. Exploratory data analysis  (on the example of MATIC/BUSD and several technical indicators:  ADOSC, NATR, TRANGE)
# 

Estimated time needed: **30** minutes

## Objectives

After completing this lab you will be able to:

*   Explore features or indicators to predict price of MATIC


## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">
<ol>
    <li>Import Data from Module</li>
    <li>Analyzing Individual Feature Patterns using Visualization</li>
    <li>Descriptive Statistical Analysis</li>
    <li>Basics of Grouping</li>
    <li>Correlation and Causation</li>
    <li>ANOVA</li>
    <li>Granger Causality Test</li>

</ol>

</div>

<hr>


### What are the main characteristics that have the most impact on the coins price?


## 1. Import Data from Module 2


#### Setup


If you run the lab locally using Anaconda, you can load the correct library and versions by uncommenting the following:


In [ ]:
#If you run the lab locally using Anaconda, you can load the correct library and versions by uncommenting the following:
#install specific version of libraries used in lab
! conda install pandas -y
! conda install numpy -y
! conda install scipy -y
! conda install seaborn -y
! conda install -c conda-forge mplfinance -y

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.dates as mpl_dates
import mplfinance as mpf
import matplotlib.pyplot as plt
import seaborn as sns
#sns.set_theme()
%matplotlib inline 
from scipy import stats
import warnings
warnings.filterwarnings("ignore")

Load the data and store it in dataframe `df`:


In [ ]:
path = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-GPXX05JFEN/labs/MATICBUSD_lab3.csv'

This dataset was hosted on IBM Cloud object. Click <a href="https://cocl.us/DA101EN_object_storage?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDA0101ENSkillsNetwork20235326-2021-01-01">HERE</a> for free storage.


\#you will need to download the dataset; if you are running locally, please comment out the following
await download(path, "auto.csv")
path="auto.csv"


In [ ]:
#read dataframe 
df = pd.read_csv(path, index_col = 0)
df.index = df.index.astype("datetime64[ns]")
main = df.copy()

In [ ]:
df.head(5)

## 2. Analyzing Individual Feature Patterns Using Visualization


To install Seaborn we use pip, the Python package manager.


Import visualization packages "Matplotlib" and "Seaborn". Don't forget about "%matplotlib inline" to plot in a Jupyter notebook.


#### How to choose the right visualization method?
<p>When visualizing individual variables, it is important to first understand what type of variable you are dealing with. This will help us find the right visualization method for that variable.</p>


In [ ]:
# list the data types for each column
print(df.dtypes)

In [ ]:
main.reset_index(inplace = True)

# Extracting Data for plotting
ohlc = main[["Ts", "Open", "High", "Low", "Close", "Volume"]].copy()

ohlc["Ts"] = pd.to_datetime(ohlc["Ts"])
ohlc.index = ohlc["Ts"]

# Resampling to 1 day
ohlc = ohlc.resample("1d").agg({
    "Open": "first",
    "High": "max",
    "Low": "min",
    "Close": "last",
    "Volume": "sum"
})

ohlc["Ts"] = ohlc.index
ohlc["Ts"] = ohlc["Ts"].apply(mpl_dates.date2num)

mpf.plot(ohlc, type="candle", 
         volume=True, 
         style="yahoo", 
         ylabel="Price", 
         xlabel="Date", 
         title="Daily Candlestick Chart of MATICBUSD")

<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h3>Question  #1:</h3>

<b>What is the data type of the column "Volume"? </b>

</div>


In [ ]:
# Write your code below and press Shift+Enter to execute 


<details><summary>Click here for the solution</summary>

```python
df['Volume'].dtypes
```

</details>


For example, we can calculate the correlation between variables  of type "int64" or "float64" using the method "corr":


In [ ]:
corr = df.corr()
corr

The diagonal elements are always one; we will study correlation more precisely Pearson correlation in-depth at the end of the notebook.
Lets take a look on corelation heatmap of our data:

In [ ]:
sns.heatmap(corr)

Now lets look on some columns more attentive:

In [ ]:
df[['Volume', 'Avg_price', 'ADOSC', 'TRANGE', 'NATR']].corr()

In [ ]:
sns.heatmap(df[['Volume', 'Avg_price', 'ADOSC', 'TRANGE', 'NATR']].corr())

<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h3> Question  #2: </h3>

<p>Find the correlation between the following columns: ADOSC, TRANGE and NATR.</p>
<p>Hint: if you would like to select those columns, use the following syntax: df[['ADOSC', 'TRANGE', 'NATR']]</p>
</div>


In [ ]:
# Write your code below and press Shift+Enter to execute 


<details><summary>Click here for the solution</summary>

```python
df[['ADOSC', 'TRANGE', 'NATR']].corr()
```

</details>


## Continuous Numerical Variables:

<p>Continuous numerical variables are variables that may contain any value within some range. They can be of type "int64" or "float64". A great way to visualize these variables is by using scatterplots with fitted lines.</p>

<p>In order to start understanding the (linear) relationship between an individual variable and the price, we can use "regplot" which plots the scatterplot plus the fitted regression line for the data.</p>


Let's see several examples of different linear relationships:


### Weak Linear Relationship

Let's find the scatterplot of "Avg_price" and "ADOSC".


In [ ]:
# Engine size as potential predictor variable of price
sns.regplot(x="Avg_price", y ="ADOSC", data=df, scatter_kws={'s':0.2})
plt.ylim(0,)

<p>Avg_price does not seem like a good predictor of the ADOSC at all since the regression line is close to horizontal. Also, the data points are very scattered and far from the fitted line, showing lots of variability. Therefore, it's not a reliable variable.</p>


We can examine the correlation between 'Avg_price' and 'ADOSC' and see that it's approximately -0.001704.


In [ ]:
df[["Avg_price", "ADOSC"]].corr()

In [ ]:
df[["Avg_price", "ADOSC"]].corr().loc["Avg_price", "ADOSC"]

### Positive Linear Relationship

Volume is a potential predictor variable of NATR. Let's find the scatterplot of "Volume" and "NATR".


In [ ]:
sns.regplot(x="NATR", y="Volume", data=df, scatter_kws={'s':0.5})

<p>As NATR goes up, the Volume goes up: this indicates an positive relationship between these two variables. NATR could potentially be a predictor of Volume.</p>


We can examine the correlation between 'Volume' and 'NATR' and see it's approximately 0.507574.


In [ ]:
df[['Volume', 'NATR']].corr().iloc[0,1]

 <div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1> Question  3 a): </h1>

<p>Find the correlation  between x="Volume" and y="TRANGE".</p>
<p>Hint: if you would like to select those columns, use the following syntax: df[["Volume","TRANGE"]].  </p>
</div>


In [ ]:
# Write your code below and press Shift+Enter to execute


<details><summary>Click here for the solution</summary>

```python


sns.regplot(x="Volume", y="TRANGE", data=df, scatter_kws={'s':0.5})

```

</details>


<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1>Question  3 b):</h1>

<p>Given the correlation results between "Volume" and "Avg_price", do you expect a linear relationship?</p> 
<p>Verify your results using the function "regplot()".</p>
</div>


In [ ]:
# Write your code below and press Shift+Enter to execute 


<details><summary>Click here for the solution</summary>

```python

#Code: 
sns.regplot(x="Volume", y="Avg_price", data=df)

```

</details>


### Categorical Variables

<p>These are variables that describe a 'characteristic' of a data unit, and are selected from a small group of categories. The categorical variables can have the type "object" or "int64". A good way to visualize categorical variables is by using boxplots.</p>


Let's look at the relationship between "Volume" and "ADOSC".


First of all, lets make some bins(like in lab2) to make analysis based on groups.

In [ ]:
bins = np.linspace(min(df["Volume"]), max(df["Volume"]), 6)
bins

In [ ]:
group_names = ['Low', 'Low-Medium', 'Medium', 'Medium-High', 'High']
df['Volume-binned'] = pd.cut(df['Volume'], bins, labels = group_names, include_lowest = True )
df[['Volume','Volume-binned']].head()

In [ ]:
sns.boxplot(x="Volume-binned", y="ADOSC", data=df)

<p>We see that the distributions of ADOSC between the Volume categories have a significant overlap, so ADOSC would not be a good predictor of Volume. Let's examine engine "NATR" and "Volume":</p>


In [ ]:
bins = np.linspace(min(df["NATR"]), max(df["NATR"]), 6)
bins

In [ ]:
group_names = ['Low', 'Low-Medium', 'Medium', 'Medium-High', 'High']
df['NATR-binned'] = pd.cut(df['NATR'], bins, labels = group_names, include_lowest = True )
df[['NATR','NATR-binned']].head()

In [ ]:
sns.boxplot(x = "NATR-binned", y = "Volume", data = df)

<p>Here we see that the distribution of Volume between this 5 NATR categories, are distinct enough to take Volume as a potential good predictor of NATR.</p>


Let's examine "Avg_price" and "NATR".


In [ ]:
bins = np.linspace(min(df["Avg_price"]), max(df["Avg_price"]), 6)
bins

In [ ]:
group_names = ['Low', 'Low-Medium', 'Medium', 'Medium-High', 'High']
df['Avg_price-binned'] = pd.cut(df['Avg_price'], bins, labels = group_names, include_lowest = True )
df[['Avg_price','Avg_price-binned']].head()

In [ ]:
# drive-wheels
sns.boxplot(x="Avg_price-binned", y="NATR", data=df)

<p>Here we see that the distribution of NATR between the different Avg_price categories differs. As such, NATR could potentially be a predictor of Avg_price.</p>


## 3. Descriptive Statistical Analysis


<p>Let's first take a look at the variables by utilizing a description method.</p>

<p>The <b>describe</b> function automatically computes basic statistics for all continuous variables. Any NaN values are automatically skipped in these statistics.</p>

This will show:

<ul>
    <li>the count of that variable</li>
    <li>the mean</li>
    <li>the standard deviation (std)</li> 
    <li>the minimum value</li>
    <li>the IQR (Interquartile Range: 25%, 50% and 75%)</li>
    <li>the maximum value</li>
<ul>


We can apply the method "describe" as follows:


In [ ]:
df.describe()

The default setting of "describe" skips variables of type category. We can apply the method "describe" on the variables of type 'category' as follows:


In [ ]:
df.describe(include=['category'])

## 4. Basics of Grouping


<p>The "groupby" method groups data by different categories. The data is grouped based on one or several variables, and analysis is performed on the individual groups.</p>

<p>For example, let's group by the variable "Volume-binned". We see that there are 5 different categories of Volume-binned.</p>


In [ ]:
df['Volume-binned'].unique()

<p>If we want to know, on average, which type of drive wheel is most valuable, we can group "Volume-binned" and then average them.</p>

<p>We can select the columns 'Volume-binned' and 'Volume', then assign it to the variable "df_group_one".</p>


In [ ]:
df_group_one = df[['Volume-binned', 'Volume']]

We can then calculate the volume for each of the different categories of data.


In [ ]:
# grouping results
df_group_one = df_group_one.groupby(['Volume-binned'], as_index = False).mean()

#set values precision as 0
pd.set_option("display.precision", 0)
df_group_one

<p>You can also group by multiple variables. For example, let's group by both 'Volume-binned' and 'Avg_price-binned'. This groups the dataframe by the unique combination of 'Volume-binned' and 'Avg_price-binned'. We can store the results in the variable 'grouped_test1'.</p>


In [ ]:
# grouping results
df_gptest = df[["Volume-binned", "Avg_price-binned", "Volume"]]
grouped_test1 = df_gptest.groupby(["Volume-binned", "Avg_price-binned"], as_index=False).mean()
grouped_test1

<p>This grouped data is much easier to visualize when it is made into a cross table. A cross table is a two-way table consisting of columns and rows. It is also known as a pivot table or a multi-dimensional table. Its greatest strength is its ability to structure, summarize and display large amounts of data. Cross tables can also be used to determine whether there is a relation between the row variable and the column variable or not.
    
    
<p>In this case, we will leave the Volume-binned variable as the rows of the table, and cross Avg_price-binned to become the columns of the table:</p>


In [ ]:
cross_table = pd.crosstab(df["Volume-binned"], df["Avg_price-binned"])
cross_table

<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1>Question 4:</h1>

<p>Use the "groupby" function to find the average "Volume" of each group based on "NATR-binned".</p>
</div>


In [ ]:
# Write your code below and press Shift+Enter to execute 
# grouping results


<details><summary>Click here for the solution</summary>

```python
# grouping results
df_gptest2 = df[['NATR-binned', 'Volume']]
grouped_test_bodystyle = df_gptest2.groupby(['NATR-binned'],as_index= False).mean()
grouped_test_bodystyle

```

</details>


If you did not import "pyplot", let's do it again.


#### Variables: Volume-binned vs. Avg_price-binned


Let's use a heat map to visualize the relationship between Volume-binned vs Avg_price-binned.

In [ ]:
#use the grouped results
plt.grid(False)
plt.pcolor(cross_table, cmap='RdBu')

plt.colorbar()
plt.show()

<p>The heatmap plots the target variable (Avg_price-binned) proportional to colour with respect to the variable 'Volume-binned' on the vertical and horizontal axis, respectively. This allows us to visualize how the Avg_price-binned is related to 'Volume-binned'.</p>

<p>The default labels convey no useful information to us. Let's change that:</p>


In [ ]:
fig, ax = plt.subplots()
plt.grid(False)
im = ax.pcolor(cross_table, cmap='RdBu')

#label names
row_labels = cross_table.columns.categories
col_labels = cross_table.index

#move ticks and labels to the center
ax.set_xticks(np.arange(cross_table.shape[1]) + 0.5, minor=False)
ax.set_yticks(np.arange(cross_table.shape[0]) + 0.5, minor=False)

#insert labels
ax.set_xticklabels(row_labels, minor=False)
ax.set_yticklabels(col_labels, minor=False)

#rotate label if too long
plt.xticks(rotation=90)

fig.colorbar(im)
plt.show()

<p>Visualization is very important in data science, and Python visualization packages provide great freedom. We will go more in-depth in a separate Python visualizations course.</p>

<p>The main question we want to answer in this module is, "What are the main characteristics which have the most impact on the coins price?".</p>

<p>To get a better measure of the important characteristics, we look at the correlation of these variables with the coins price. In other words: how is the coin price dependent on this variable?</p>


## 5. Correlation and Causation


<p><b>Correlation</b>: a measure of the extent of interdependence between variables.</p>

<p><b>Causation</b>: the relationship between cause and effect between two variables.</p>

<p>It is important to know the difference between these two. Correlation does not imply causation. Determining correlation is much simpler  the determining causation as causation may require independent experimentation.</p>


<p><b>Pearson Correlation</b></p>
<p>The Pearson Correlation measures the linear dependence between two variables X and Y.</p>
<p>The resulting coefficient is a value between -1 and 1 inclusive, where:</p>
<ul>
    <li><b>1</b>: Perfect positive linear correlation.</li>
    <li><b>0</b>: No linear correlation, the two variables most likely do not affect each other.</li>
    <li><b>-1</b>: Perfect negative linear correlation.</li>
</ul>


<p>Pearson Correlation is the default method of the function "corr". Like before, we can calculate the Pearson Correlation of the of the 'int64' or 'float64'  variables.</p>


In [ ]:
#set values precision as 6
pd.set_option("display.precision", 6)
df.corr()

Sometimes we would like to know the significant of the correlation estimate.


<b>P-value</b>

<p>What is this P-value? The P-value is the probability value that the correlation between these two variables is statistically significant. Normally, we choose a significance level of 0.05, which means that we are 95% confident that the correlation between the variables is significant.</p>

By convention, when the

<ul>
    <li>p-value is $<$ 0.001: we say there is strong evidence that the correlation is significant.</li>
    <li>the p-value is $<$ 0.05: there is moderate evidence that the correlation is significant.</li>
    <li>the p-value is $<$ 0.1: there is weak evidence that the correlation is significant.</li>
    <li>the p-value is $>$ 0.1: there is no evidence that the correlation is significant.</li>
</ul>


We can obtain this information using  "stats" module in the "scipy"  library.


### Volume vs. ADOSC


Let's calculate the  Pearson Correlation Coefficient and P-value of 'Volume' and 'ADOSC'.


In [ ]:
pearson_coef, p_value = stats.pearsonr(df['Volume'], df['ADOSC'])
print("The Pearson Correlation Coefficient is", round(pearson_coef, 2), " with a P-value of P =", round(p_value, 2))  

#### Conclusion:
<p>Since the p-value is $<$ 0.001, the correlation between Volume and ADOSC is statistically significant, although the linear relationship is weak (~0.1).</p>


### Volume vs. TRANGE


Let's calculate the  Pearson Correlation Coefficient and P-value of 'Volume' and 'TRANGE'.


In [ ]:
pearson_coef, p_value = stats.pearsonr(df['Volume'], df['TRANGE'])
print("The Pearson Correlation Coefficient is", round(pearson_coef, 2), " with a P-value of P = ", round(p_value, 2))  

#### Conclusion:

<p>Since the p-value is $<$ 0.001, the correlation between Volume and TRANGE is statistically significant, and the linear relationship is not strong (~0.25).</p>


### Volume vs. NATR

Let's calculate the  Pearson Correlation Coefficient and P-value of 'Volume' and 'NATR'.


In [ ]:
pearson_coef, p_value = stats.pearsonr(df['Volume'], df['NATR'])
print("The Pearson Correlation Coefficient is", round(pearson_coef, 2), " with a P-value of P = ", round(p_value, 2) )  

#### Conclusion:
<p>Since the p-value is $<$ 0.001, the correlation between Volume and NATR is statistically significant, and the linear relationship is quiet strong (~0.51).</p>


### Avg_price vs. Volume


Let's calculate the Pearson Correlation Coefficient and P-value of 'Avg_price' and 'Volume':


In [ ]:
pearson_coef, p_value = stats.pearsonr(df['Avg_price'], df['Volume'])
print("The Pearson Correlation Coefficient is", round(pearson_coef, 2), " with a P-value of P = ", round(p_value, 2) )

#### Conclusion:

Since the p-value is < 0.001, the correlation between Avg_price and Volume is statistically significant, and the linear relationship is very weak (~ -0.02).


### Avg_price vs. ADOSC


Let's calculate the Pearson Correlation Coefficient and P-value of 'Avg_price' and 'ADOSC':


In [ ]:
pearson_coef, p_value = stats.pearsonr(df['Avg_price'], df['ADOSC'])
print("The Pearson Correlation Coefficient is", round(pearson_coef, 2), " with a P-value of P = ", round(p_value, 2) ) 

#### Conclusion:
<p>Since the p-value is $>$ 0.05, the correlation between Avg_price and ADOSC is not statistically significant</p>


### Avg_price vs. TRANGE

Let's calculate the Pearson Correlation Coefficient and P-value of 'Avg_price' and 'TRANGE':


In [ ]:
pearson_coef, p_value = stats.pearsonr(df['Avg_price'], df['TRANGE'])
print("The Pearson Correlation Coefficient is", round(pearson_coef, 2), " with a P-value of P = ", round(p_value, 2) )

#### Conclusion:

<p>Since the p-value is $<$ 0.001, the correlation between Avg_price and TRANGE is statistically significant, and the linear relationship is weak (~0.08).</p>


### Avg_price vs. NATR


Let's calculate the  Pearson Correlation Coefficient and P-value of 'Avg_price' and 'NATR':


In [ ]:
pearson_coef, p_value = stats.pearsonr(df['Avg_price'], df['NATR'])
print("The Pearson Correlation Coefficient is", round(pearson_coef, 2), " with a P-value of P =  ", round(p_value, 2) ) 

#### Conclusion:
<p>Since the p-value is $<$ 0.001, the correlation between Avg_price and NATR is statistically significant, but the linear relationship is very weak (~0.09).</p>


## 6. ANOVA


### ANOVA: Analysis of Variance
<p>The Analysis of Variance  (ANOVA) is a statistical method used to test whether there are significant differences between the means of two or more groups. ANOVA returns two parameters:</p>

<p><b>F-test score</b>: ANOVA assumes the means of all groups are the same, calculates how much the actual means deviate from the assumption, and reports it as the F-test score. A larger score means there is a larger difference between the means.</p>

<p><b>P-value</b>:  P-value tells how statistically significant our calculated score value is.</p>

<p>If our price variable is strongly correlated with the variable we are analyzing, we expect ANOVA to return a sizeable F-test score and a small p-value.</p>


## Volume

<p>Since ANOVA analyzes the difference between different groups of the same variable, the groupby function will come in handy. Because the ANOVA algorithm averages the data automatically, we do not need to take the average before hand.</p>

<p>To see if different types of 'Volume-binned' impact  'Volume', we group the data.</p>


In [ ]:
grouped_test2=df_gptest[['Volume-binned', 'Volume']].groupby(['Volume-binned'])
grouped_test2.head(2)

In [ ]:
df_gptest

We can obtain the values of the method group using the method "get_group".


In [ ]:
grouped_test2.get_group('Low')['Volume']

We can use the function 'f_oneway' in the module 'stats' to obtain the <b>F-test score</b> and <b>P-value</b>.


In [ ]:
# ANOVA
f_val, p_val = stats.f_oneway(grouped_test2.get_group('Low')['Volume'], grouped_test2.get_group('Medium')['Volume'], grouped_test2.get_group('High')['Volume'])  
 
print( "ANOVA results: F=", round(f_val, 2), ", P =", round(p_val, 2))   

This is a great result with a large F-test score showing a strong correlation and a P-value of almost 0 implying almost certain statistical significance. But does this mean all three tested groups are all this highly correlated?

Let's examine them separately.


#### Low and Medium


In [ ]:
f_val, p_val = stats.f_oneway(grouped_test2.get_group('Low')['Volume'], grouped_test2.get_group('Medium')['Volume'])  
 
print( "ANOVA results: F=", round(f_val, 2), ", P =", round(p_val, 2))  

Let's examine the other groups.


#### Low and High


In [ ]:
f_val, p_val = stats.f_oneway(grouped_test2.get_group('Low')['Volume'], grouped_test2.get_group('High')['Volume'])  
   
print( "ANOVA results: F=", round(f_val, 2), ", P =", round(p_val, 2))    

#### Medium and High


In [ ]:
f_val, p_val = stats.f_oneway(grouped_test2.get_group('Medium')['Volume'], grouped_test2.get_group('High')['Volume'])  
   
print( "ANOVA results: F=", round(f_val, 2), ", P =", round(p_val, 2))    

## 7. Granger Causality Test
The Granger causality test is a statistical hypothesis test for determining whether one time series is useful in forecasting another, first proposed in 1969. Ordinarily, regressions reflect "mere" correlations, but Clive Granger argued that causality in economics could be tested for by measuring the ability to predict the future values of a time series using prior values of another time series. Since the question of "true causality" is deeply philosophical, and because of the post hoc ergo propter hoc fallacy of assuming that one thing preceding another can be used as a proof of causation, econometricians assert that the Granger test finds only "predictive causality". Using the term "causality" alone is a misnomer, as Granger-causality is better described as "precedence", or, as Granger himself later claimed in 1977, "temporally related". Rather than testing whether X causes Y, the Granger causality tests whether X forecasts Y.

In [ ]:
df[["Avg_price", "Volume"]].head

In [ ]:
from statsmodels.tsa.stattools import grangercausalitytests
#Check Granger Causality of all possible combinations of the Time series
def causation_matrix(data, maxlag, variables, test = 'ssr_chi2test', verbose = False):
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns = variables, index = variables)
    for column in df.columns:
        for index in df.index:
            #find gct result
            test_result = grangercausalitytests(data[[index, column]], maxlag = maxlag, verbose = False)
            #find p-values
            p_values = [round(test_result[i + 1][0][test][1], 4) for i in range(maxlag)]
            if verbose:
                print(f'Y = {r}, X = {c}, P Values = {p_values}')
            min_p_value = np.min(p_values)
            df.loc[index, column] = min_p_value
    #show result
    df.columns = [var + '_X' for var in variables]
    df.index = [var + '_Y' for var in variables]
    return df


In [ ]:
#create new df with data about 2 coins
test_df = df[["Avg_price", "Volume"]]

x = df.index
y1 = test_df["Avg_price"]
y2 = test_df["Volume"]
    
fig, ax1 = plt.subplots(1, 1, figsize = (16,9), dpi = 80)
ax1.plot(x, y1, color = 'tab:red')
    
ax2 = ax1.twinx()
ax2.plot(x, y2, color = 'tab:blue')
    
ax1.set_xlabel("ts", fontsize = 20)
ax1.tick_params(axis = 'x', rotation = 0, labelsize = 12)
ax1.set_ylabel("Avg_price", color = 'tab:red', fontsize = 20)
ax1.tick_params(axis = 'y', rotation = 0, labelcolor = 'tab:red')
ax1.grid(alpha=.4)
    
from matplotlib import pyplot as plt, dates
ax2.set_ylabel("Volume", color = 'tab:blue', fontsize = 20)
ax2.tick_params(axis = 'y', rotation = 0, labelcolor = 'tab:blue')
ax2.xaxis.set_major_formatter(dates.DateFormatter('%y-%m-%d'))
fig.autofmt_xdate(rotation=45)
#ax2.set_xticks(np.arange(0, len(x), 60))
ax2.set_xticklabels(x[::60], rotation = 90, fontdict = {'fontsize':10})
ax2.set_title("Visualizing Leading Indicator Phenomenon", fontsize = 22)
    
fig.tight_layout()
plt.show()


In [ ]:
causation_matrix(df[["Avg_price", "TRANGE"]], 2, variables = df[["Avg_price", "TRANGE"]].columns)

How to interpret the p-values?

Assuming a significance level of 0.05, if the p-value is lesser than 0.05, then we do NOT reject the null hypothesis that X does NOT granger cause Y.

So, in the above table, the p-value for Avg_price_X and TRANGE_X is 0.0. So we reject the null hypothesis and conclude that (Avg_price) granger causes (TRANGE).

However the p-value for Avg_price_X and TRANGE_Y is 0.0133.

Since the p-value is less than 0.05, we can reject the null hypothesis. That is, "Avg_price_X" can predictive of "TRANGE_Y".

Let's calculate Granger Causality Test for other coins:

In [ ]:
print("Avg_price(Volume) = ", round(causation_matrix(df[["Avg_price", "Volume"]], 1, variables = df[["Avg_price", "Volume"]].columns).iloc[0,1], 3))

In [ ]:
print("Avg_price(ADOSC) = ", round(causation_matrix(df[["Avg_price", "ADOSC"]], 1, variables = df[["Avg_price", "ADOSC"]].columns).iloc[0,1], 3))

In [ ]:
print("Avg_price(NATR) = ", round(causation_matrix(df[["Avg_price", "NATR"]], 1, variables = df[["Avg_price", "NATR"]].columns).iloc[0,1], 3))

In [ ]:
df.to_csv("MATICBUSD_lab4.csv")

## Conclusion:
We now have a better idea of what our data looks like and which variables are more related to our main currency MATICBUSD.
But all our relations is realy weak, so we cant use that variable to predict price of our coin. That`s mean we need to find a relations of our coin with other coins for example BTC.
As we now move into building machine learning models to automate our analysis, feeding the model with variables that meaningfully affect our target variable will improve our model's prediction performance.

# **Thank you for completing Lab 3!**

## Authors

<a href="https://author.skills.network/instructors/oleh_lozovyi?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkGuidedProjectsIBMSkillsNetworkGPXX0QGDEN2306-2023-01-01">Oleh Lozovyi</a>

<a href="https://author.skills.network/instructors/yaroslav_vyklyuk_2?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkGuidedProjectsIBMSkillsNetworkGPXX0QGDEN2306-2023-01-01">Prof. Yaroslav Vyklyuk, DrSc, PhD</a>

<a href="https://author.skills.network/instructors/mariya_fleychuk?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkGuidedProjectsIBMSkillsNetworkGPXX0QGDEN2306-2023-01-01">Prof. Mariya Fleychuk, DrSc, PhD</a>

<a href="https://www.linkedin.com/in/joseph-s-50398b136/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDA0101ENSkillsNetwork20235326-2021-01-01">Joseph Santarcangelo</a>


<a href="https://www.linkedin.com/in/fiorellawever/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDA0101ENSkillsNetwork20235326-2021-01-01">Fiorella Wenver</a>

<a href="https:// https://www.linkedin.com/in/yi-leng-yao-84451275/ " target="_blank" >Yi Yao</a>


## Change Log

| Date (YYYY-MM-DD) | Version | Changed By   | Change Description                                         |
| ----------------- | ------- | -------------| ---------------------------------------------------------- |
|     2023-03-15    |   1.0   | Oleh Lozovyi | Lab created                                                |

<hr>

## <h3 align="center"> © IBM Corporation 2023. All rights reserved. <h3/>



